In [ ]:
import statsmodels.api as sm
import numpy as np
import pandas as pd

import pdb
import os
import matplotlib.pyplot as plt
import seaborn as sns


curr_dir = '/user_data/vayzenbe/GitHub_Repos/docnet/'

: 

In [ ]:
rois = ['lLO_toolloc', 'lPFS_toolloc', 'rLO_toolloc', 'rPFS_toolloc','rPPC_spaceloc', 'rAPC_spaceloc']
rois = ['PPC_spaceloc', 'APC_spaceloc', 'LO_toolloc']
roi_names = ['PPC', 'APC','LO']
file_suf = "_supp"
models = ['skel', 'cornet_s','gbj']

rdms = pd.read_csv(f'{curr_dir}/results/rsa/all_rdms{file_suf}.csv')

rdms = (rdms-rdms.mean())/rdms.std()
                
iter_num = 5000
alpha = .05

: 

In [ ]:
def calc_rsa(roi, model_data, covs):
    # Pull out the data
    
    X = model_data[covs]
    y = roi
    model = sm.OLS(y, X).fit()
    
    
    return model.rsquared, model.params, model.pvalues

: 

In [7]:
def calc_vpa_unique(roi, model_data, covs):
    full = sm.OLS(roi, model_data[covs]).fit()
    unique_var = []
    for cv in covs:
        partial_cov1 = covs[:]
        partial_cov1.remove(cv) 
        partial = sm.OLS(roi, model_data[partial_cov]).fit()

        unique_var.append(full.rsquared - partial.rsquared)
        

In [ ]:
def calc_vpa_shared(roi, model_data, covs):
    full = sm.OLS(roi, model_data[covs]).fit()

    for cv1n in range(0,len(covs)):
        for cv2n in range(cv1n+1, len(covs)):
            #set up each partial cov
            cov_ij = covs[:]
            cov_ij.remove(covs[cv1n])
            
            cov_jk = covs[:]
            cov_jk.remove(covs[cv2n])

            cov_k = covs[:]
            cov_k.remove(covs[cv1n])



    

In [ ]:
summary_df = pd.DataFrame(columns = ['roi', 'R'] + \
     [m + '_beta' for m in models] + [m + '_unique' for m in models] + ['skel_gbj_shared','skel_cornet_shared'] + \
        [m + '_beta_ci' for m in models] + [m + '_unique_ci' for m in models] + ['skel_gbj_shared_ci','skel_cornet_shared_ci']  )

: 

In [4]:
#rois = ['PPC_spaceloc']

summary_df = pd.DataFrame(columns = ['roi', 'R'] + \
     [m + '_beta' for m in models] + [m + '_se' for m in models] + [m + '_beta_ci_low' for m in models]+ [m + '_beta_ci_high' for m in models])
for rr in rois:
    for lr in ['l','r']:
        r_square, mean_beta,p_val = calc_rsa(rdms[f'{lr}{rr}'],rdms,models)
        
        beta_samples = []
        for itn in range(0,iter_num):
            
            temp_rdm = rdms.sample(frac = 1, replace = True, ignore_index = True)#sample with replacement
            temp_rsquare, temp_beta, temp_p = calc_rsa(temp_rdm[f'{lr}{rr}'],temp_rdm,models) 
            beta_samples.append(temp_beta.to_list())
        
        beta_samples = np.array(beta_samples)
        ci_low = []
        ci_high = []
        se = []
        for mm in enumerate(models):
            se.append(np.std(beta_samples[:,mm[0]]))
            ci_low.append(np.percentile(beta_samples[:,mm[0]], alpha/2*100))
            ci_high.append(np.percentile(beta_samples[:,mm[0]], 100-alpha/2*100))

        #beta_samples = pd.DataFrame(np.array(beta_samples))
        
        roi_data = [f'{lr}{rr}', r_square] + mean_beta.to_list() + se + ci_low + ci_high
        
        summary_df = summary_df.append(pd.Series(roi_data, index =summary_df.columns), ignore_index = True )
        

summary_df.to_csv(f'/user_data/vayzenbe/GitHub_Repos/docnet/results/rsa/RSA_betas{file_suf}.csv', index = False)
                                        
        

In [10]:
summary_df

,roi,R,skel_beta,cornet_s_beta,gbj_beta,skel_beta_ci_low,cornet_s_beta_ci_low,gbj_beta_ci_low,skel_beta_ci_high,cornet_s_beta_ci_high,gbj_beta_ci_high
0,lPPC_spaceloc,0.009058,0.024122,-0.096930,-0.029544,-0.129636,-0.253917,-0.147016,0.187321,0.056564,0.091952
1,rPPC_spaceloc,0.109417,0.328781,-0.015079,0.039416,0.183774,-0.170473,-0.063097,0.482736,0.140170,0.141496
2,lAPC_spaceloc,0.005480,0.039272,-0.040063,-0.064643,-0.119787,-0.198105,-0.181976,0.199427,0.120862,0.047430
3,rAPC_spaceloc,0.011835,0.141851,-0.069428,-0.002464,-0.013980,-0.225868,-0.125486,0.297768,0.088868,0.124690
4,lAPC_toolloc,0.005237,0.037737,0.038940,0.007230,-0.102773,-0.115649,-0.118753,0.184805,0.191918,0.126423
5,rAPC_toolloc,0.001761,0.052109,-0.031525,-0.019359,-0.099581,-0.175704,-0.137711,0.205548,0.117101,0.099099
6,lLO_toolloc,0.077654,0.171314,0.135390,-0.004607,0.037346,-0.007191,-0.129488,0.305375,0.284157,0.119026
7,rLO_toolloc,0.026605,0.042252,0.129312,0.008301,-0.108770,-0.025940,-0.114210,0.191648,0.284614,0.128506
8,lPFS_toolloc,0.013008,0.138296,-0.060032,0.022474,-0.010610,-0.221601,-0.105090,0.292486,0.099815,0.145049
9,rPFS_toolloc,0.011690,-0.100237,0.140995,0.011056,-0.247350,-0.015729,-0.094483,0.044339,0.299899,0.118807


In [51]:
temp_rdm

,cornet_s,gbj,lAPC_depthloc,lAPC_distloc,lAPC_spaceloc,lAPC_toolloc,lLO_toolloc,lPFS_toolloc,lPPC_depthloc,lPPC_distloc,...,rAPC_toolloc,rLO_toolloc,rPFS_toolloc,rPPC_depthloc,rPPC_distloc,rPPC_spaceloc,rPPC_toolloc,skel,stim1,stim2
0,0.463334,1.046686,-0.321047,1.398512,1.302625,0.558480,0.565399,1.294559,1.720760,0.571122,...,0.983632,-1.496186,0.720510,0.757611,0.202483,0.966956,1.680138,0.271033,NaN,NaN
1,0.983339,0.803459,-0.365248,-0.708930,1.123348,-0.805581,-1.175210,1.269310,-0.088837,-0.772579,...,-0.425628,-0.136085,-1.086228,1.754780,0.018770,-1.042444,-1.384524,0.154715,NaN,NaN
2,-0.366486,0.561132,0.154326,1.339248,-0.117399,1.048624,-0.230033,-0.059884,0.911725,1.210137,...,-0.186628,-0.740318,-0.409945,-0.657430,-0.092109,-1.472017,-0.100819,-0.751032,NaN,NaN
3,-0.728810,-0.131681,-0.651904,-2.363112,0.085128,-1.440785,-1.196266,-1.162645,0.734317,-1.068978,...,-0.557136,-1.849758,0.558660,-0.972559,-2.945733,-1.780894,-1.111600,-0.221644,NaN,NaN
4,-1.429106,0.029271,-0.154188,-0.173885,-0.889897,0.644629,0.040685,0.948490,1.248838,-0.045952,...,-1.335911,0.713246,0.983660,0.236888,-0.609599,0.511780,-0.308249,-1.495132,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,-0.314912,0.463122,1.460592,0.587067,0.887995,0.010191,0.353667,-1.826035,-0.273789,0.739992,...,-0.871752,-2.701835,-2.642752,0.461080,0.244499,-1.219094,-0.153179,1.255177,NaN,NaN
296,0.932083,-0.344740,-0.348873,2.415502,2.371269,1.400118,0.899982,2.198895,0.260037,1.510607,...,1.415843,1.000540,1.567021,-0.691269,0.666125,0.862541,0.406208,-0.062946,NaN,NaN
297,-1.246237,-0.582886,-0.611236,-0.632515,-1.773773,-1.013681,-1.059035,0.262896,-0.357490,1.449423,...,-0.445265,0.674387,0.782809,-0.952766,0.253244,-0.532980,-0.240438,-1.538472,NaN,NaN
298,-0.055174,0.917205,0.649902,2.151090,1.373655,0.923867,-0.326375,-0.497001,0.065984,1.210359,...,1.025611,-0.699121,-0.448534,0.041313,1.535235,-0.635865,0.852832,-0.495249,NaN,NaN


In [49]:
beta_samples = pd.DataFrame(np.array(beta_samples))

In [55]:
roi_data = [f'{lr}{rr}'] + mean_beta.to_list() + beta_samples.sem(axis = 0).to_list()
roi_data

['lPPC_spaceloc',
 0.02412248540437869,
 -0.0969298193274901,
 -0.02954411521415691,
 0.020812338815115152,
 0.031539649088171244,
 0.01922661454536177]

In [59]:
summary_df = pd.DataFrame(columns = ['roi'] + models + [s + '_se' for s in models] )
summary_df = summary_df.append(pd.Series(roi_data, index =summary_df.columns), ignore_index = True )
summary_df

,roi,skel,cornet_s,gbj,skel_se,cornet_s_se,gbj_se
0,lPPC_spaceloc,0.024122,-0.09693,-0.029544,0.020812,0.03154,0.019227


In [57]:
summary_df

,0,cornet_s,cornet_s_se,gbj,gbj_se,roi,skel,skel_se
0,lPPC_spaceloc,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.024122,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-0.09693,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.029544,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.020812,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.03154,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,0.019227,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
temp_rdm

108    0.951908
155    0.557901
152   -0.061268
265   -1.600217
71    -1.584540
         ...   
261   -2.053389
83     1.471191
92    -0.238132
73    -0.900139
147   -1.038192
Name: rAPC_spaceloc, Length: 300, dtype: float64